#BERTopic on Quran Urdu Translations
In this notebook I have implemented Topic Modeling on Urdu translations based topic modelling technique BERTopic.

Shaista Zulfiqar

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0




#Installing required dependencies
**One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.**

In [ ]:
!pip install bertopic
!pip install -U sentence-transformers
!pip install urduhack

In [ ]:
!pip install --upgrade keras tensorflow-addons


# Importing required dependencies
We will import numpy, pandas and re, bertopic, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data. gensim library is used to get coherence score and train LDA. bertopic is used to train bertopic on our Quran-UTM dataset with using pretrained language models Multilingial MiniLM

In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from urduhack.normalization import remove_diacritics
from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

##DataFrame




In [ ]:

junapd = pd.read_table("provide your data path here", header=None, encoding='utf-8')

print(junapd.head(5))

stopspd=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List'])

stopspd

                                                   0
0              1|1|اللہ کے نام سے جو رحمان و رحیم ہے
1  1|2|تعریف اللہ ہی کے لیے ہے جو تمام کائنات کا ...
2                              1|3|رحمان اور رحیم ہے
3                             1|4|روز جزا کا مالک ہے
4  1|5|ہم تیری ہی عبادت کرتے ہیں اور تجھی سے مدد ...


,List
0,آ
1,آؤ
2,آو
3,آئی
4,آئیں
...,...
641,یعنی
642,یہ
643,یہاں
644,یہی


## Cleaning of Data
After collecting the eight Quran Urdu translations, we observed these translation had some irrelevant information, such as metadata, punctuation, and diacritics. So we cleaned the translations for topic modeling

Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling. Stopwords are removed in post preprocessing phase

In [ ]:
import re
#Remove Urdu Punctuation
def remove_urdu_punctuation(text):
    pattern = r'[؛؟،٫٬‘’“”«»!"٪&\'\*\+,-./:;<=>؟@^_`()[]{|}~]'

    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text


In [ ]:
# Remove trailing metadata lines
index_to_drop = junapd[junapd[0].str.startswith("#")].index
junapd.drop(index_to_drop, inplace=True)

# Remove verse reference from every line
junapd[0] = junapd[0].str.replace(r'\d+\|\d+\|', '', regex=True)

# Remove diacritics
junapd[0] = junapd[0].apply(remove_diacritics)

# Remove punctuation
junapd[0] = junapd[0].apply(remove_urdu_punctuation)

# Reset index
junapd.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(junapd.head())



Cleaned DataFrame:
                                                   0
0                  اللہ کے نام سے جو رحمان و رحیم ہے
1    تعریف اللہ ہی کے لیے ہے جو تمام کائنات کا رب ہے
2                                  رحمان اور رحیم ہے
3                                 روز جزا کا مالک ہے
4  ہم تیری ہی عبادت کرتے ہیں اور تجھی سے مدد مانگ...


In [ ]:
def remove_nonbreaking_space(text):
    return re.sub(r'\xa0', ' ', text)

junapd[0] = junapd[0].apply(remove_nonbreaking_space)

In [ ]:
# Save the cleaned DataFrame to a text file
file_path = 'cleaned_Maududi_text.txt'
junapd[0].to_csv(file_path, sep='\n', index=False, header=False, encoding='utf-8')

print(f'Text has been saved to {file_path}')

Text has been saved to cleaned_Maududi_text.txt


In [ ]:
# Convert the DataFrame to a list of strings
data = junapd[0].tolist()

In [ ]:
print(len(data))

6236


In [ ]:
print(data[56:90])

['یاد کرو وہ وقت، جب ہم نے سمندر پھاڑ کر تمہارے لیے راستہ بنایا، پھر اس میں سے تمہیں بخیریت گزروا دیا، پھر وہیں تمہاری آنکھوں کے سامنے فرعونوں کو غرقاب کیا', 'یاد کرو، جب ہم نے موسیؑ کو چالیس شبانہ روز کی قرارداد پر بلایا، تو اس کے پیچھے تم بچھڑے کو اپنا معبود بنا بیٹھے اس وقت تم نے بڑی زیادتی کی تھی', 'مگر اس پر بھی ہم نے تمہیں معاف کر دیا کہ شاید اب تم شکر گزار بنو', 'یاد کرو کہ (ٹھیک اس وقت جب تم یہ ظلم کر رہے تھے) ہم نے موسیؑ کو کتاب اور فرقان عطا کی تاکہ تم اس کے ذریعے سے سیدھا راستہ پاسکو', 'یاد کرو جب موسیؑ (یہ نعمت لیتے ہوئے پلٹا، تو اس) نے اپنی قوم سے کہا کہ "لوگو، تم نے بچھڑے کو معبود بنا کر اپنے اوپر سخت ظلم کیا ہے، لہذا تم لوگ اپنے خالق کے حضور توبہ کرو اور اپنی جانوں کو ہلاک کرو، اسی میں تمہارے خالق کے نزدیک تمہاری بہتری ہے" اس وقت تمہارے خالق نے تمہاری توبہ قبول کر لی کہ وہ بڑا معاف کرنے والا اور رحم فرمانے والا ہے', 'یاد کرو جب تم نے موسیؑ سے کہا تھا کہ ہم تمہارے کہنے کا ہرگز یقین نہ کریں گے، جب تک کہ اپنی آنکھوں سے علانیہ خدا کو (تم سے کلام کرتے) نہ دیکھ لیں اس وقت تمہا

# BERTopic Training
The default  bertopic embedding model is paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual". We take MiniLM Model from [sentence-tranformer](https://www.sbert.net/docs/pretrained_models.html) and create custom document embedding and passed it to the bertopic model for training.

In [ ]:
#create custom embedding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(data, show_progress_bar=True)
print(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

[[-0.05742984  0.69452333 -0.30106208 ... -0.04793055  0.13754316
   0.0710004 ]
 [-0.0586119   0.5248769  -0.03704967 ... -0.15689774  0.14192663
   0.00624371]
 [ 0.02151142  0.47360063 -0.15387863 ... -0.20607708  0.27051833
  -0.04277751]
 ...
 [-0.00938364  0.05366365  0.0013283  ... -0.05590227  0.25281447
   0.13874325]
 [-0.03382144  0.00611624 -0.11566864 ... -0.04479951  0.1560542
   0.0047373 ]
 [ 0.04968645  0.10207345 -0.03612753 ... -0.16588512  0.13069567
  -0.11995934]]


In [ ]:
stop_words_list = stopspd['List'].tolist()

In [ ]:
#pass vectorizer_model to bertopic for stopwords removal
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words= stop_words_list)


In [ ]:
#UMAP for dimention reduction
from umap import UMAP
dim_model = UMAP(n_components=4, random_state=42)

In [ ]:
# # #KMeans used for clustering
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=5, random_state=42)

In [ ]:
np.random.seed(42)

In [ ]:
topic_model = BERTopic(language="urdu", low_memory=True ,calculate_probabilities=True,  top_n_words=10,hdbscan_model=cluster_model, umap_model=dim_model, verbose=True, vectorizer_model=vectorizer_model)

In [ ]:
#Fit documents in bertopic
topics, probs = topic_model.fit_transform(data,embeddings)

2024-06-21 07:53:41,821 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-21 07:54:27,001 - BERTopic - Dimensionality - Completed ✓
2024-06-21 07:54:27,004 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-21 07:54:27,601 - BERTopic - Cluster - Completed ✓
2024-06-21 07:54:27,617 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-21 07:54:27,955 - BERTopic - Representation - Completed ✓


In [ ]:
print(probs)

None


In [ ]:
#topics that assign to each document
print(topics)

[2, 1, 1, 0, 1, 0, 0, 0, 2, 1, 2, 1, 0, 2, 2, 2, 2, 0, 0, 1, 1, 2, 0, 2, 0, 2, 2, 1, 2, 1, 0, 1, 2, 2, 2, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 3, 3, 3, 0, 3, 3, 3, 0, 1, 0, 0, 3, 3, 2, 1, 2, 0, 0, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 1, 3, 1, 0, 0, 1, 2, 2, 2, 2, 1, 1, 2, 2, 0, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 1, 1, 0, 1, 2, 2, 1, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 0, 2, 1, 2, 1, 1, 1, 2, 2, 2, 1, 0, 2, 2, 0, 2, 1, 2, 2, 1, 1, 0, 1, 0, 2, 0, 2, 1, 2, 2, 1, 2, 2, 2, 0, 2, 2, 1, 0, 0, 2, 2, 1, 0, 2, 1, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 0, 2, 2, 2, 1, 2, 1, 0, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 0, 

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
2,0,2470
1,1,1764
0,2,1719
3,3,268
4,4,15


In [ ]:
document_topics = topic_model.get_topics()

In [ ]:
#topics with score
print(document_topics)

{0: [('لوگ', 0.04362055171922223), ('لوگوں', 0.03575660315403548), ('دن', 0.03448786286149961), ('وقت', 0.029945440531769476), ('عذاب', 0.02770599228157185), ('زمین', 0.026054287526549123), ('پیدا', 0.02416433793198817), ('روز', 0.021634081688631346), ('چیز', 0.021480222253272098), ('دیکھ', 0.021265572454587208)], 1: [('رب', 0.08556558840943448), ('لوگوں', 0.044873143516998865), ('لوگ', 0.04217119185443228), ('ایمان', 0.040106600339538695), ('خدا', 0.0370506119880038), ('زمین', 0.03363548775168101), ('جن', 0.028590665778745996), ('اللہ', 0.025671245378825452), ('بات', 0.02546159690898182), ('کتاب', 0.024131930958262934)], 2: [('اللہ', 0.1510758018795758), ('لوگ', 0.04277051711153894), ('لوگوں', 0.03828851652518962), ('ایمان', 0.034099353386142346), ('جن', 0.024134825975673412), ('رسول', 0.02342070266669578), ('حق', 0.02215088242822512), ('نبی', 0.019946884488641667), ('رب', 0.019696439389604598), ('بات', 0.0195319794635111)], 3: [('موسی', 0.23439763197046948), ('فرعون', 0.1341825460483

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2470,0_لوگ_لوگوں_دن_وقت,"[لوگ, لوگوں, دن, وقت, عذاب, زمین, پیدا, روز, چ...",[ایک دن ضرور جمع کیے جانے والے ہیں جس کا وقت م...
1,1,1764,1_رب_لوگوں_لوگ_ایمان,"[رب, لوگوں, لوگ, ایمان, خدا, زمین, جن, اللہ, ب...",[پھر ان لوگوں کو کیا ہو گیا ہے کہ یہ ایمان نہی...
2,2,1719,2_اللہ_لوگ_لوگوں_ایمان,"[اللہ, لوگ, لوگوں, ایمان, جن, رسول, حق, نبی, ر...",[تمہارے رفیق تو حقیقت میں صرف اللہ اور اللہ کا...
3,3,268,3_موسی_فرعون_قوم_نوح,"[موسی, فرعون, قوم, نوح, رب, اسرائیل, بنی, لوگو...","[موسیؑ اور ہارونؑ کے رب کو"", فرعون نے کہا "" اچ..."
4,4,15,4_کن_انعامات_جھٹلاؤ_رب,"[کن, انعامات, جھٹلاؤ, رب, , , , , , ]","[اپنے رب کے کن کن انعامات کو تم جھٹلاؤ گے؟, اپ..."


In [ ]:
topic_distr, _ = topic_model.approximate_distribution(data, window=3, min_similarity=0.01)

100%|██████████| 7/7 [00:01<00:00,  3.51it/s]


In [ ]:
print(topic_distr)

[[0.02305535 0.42196541 0.53711591 0.01786332 0.        ]
 [0.02944889 0.31445315 0.40471106 0.13164217 0.11974473]
 [0.         0.64008811 0.35991189 0.         0.        ]
 ...
 [0.52518974 0.20867778 0.26613248 0.         0.        ]
 [0.27515313 0.29766688 0.28179637 0.14538362 0.        ]
 [0.42988238 0.31669565 0.25342197 0.         0.        ]]


In [ ]:
topic_model.visualize_distribution(topic_distr[0], width=600,height=600, title="Topic Probability Distribution")

# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score
To evaluate the model topics coherence we use [Gensim](https://radimrehurek.com/gensim/models/coherencemodel.html) library

In [ ]:
texts = [[word for word in str(document).split() if word not in stop_words_list] for document in data] #if word not in stop_words_list
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
topics_bert=[]
for i in topic_model.get_topics():
  row=[]
  topic= topic_model.get_topic(i)
  for word in topic:
     row.append(word[0])
  topics_bert.append(row)

In [ ]:
print(topics_bert)

[['لوگ', 'لوگوں', 'دن', 'وقت', 'عذاب', 'زمین', 'پیدا', 'روز', 'چیز', 'دیکھ'], ['رب', 'لوگوں', 'لوگ', 'ایمان', 'خدا', 'زمین', 'جن', 'اللہ', 'بات', 'کتاب'], ['اللہ', 'لوگ', 'لوگوں', 'ایمان', 'جن', 'رسول', 'حق', 'نبی', 'رب', 'بات'], ['موسی', 'فرعون', 'قوم', 'نوح', 'رب', 'اسرائیل', 'بنی', 'لوگوں', 'ہارون', 'بنا'], ['کن', 'انعامات', 'جھٹلاؤ', 'رب', '', '', '', '', '', '']]


In [ ]:
# compute Coherence Score CV

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_v')
coherence = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence)

In [ ]:
# compute Coherence Score NPMI

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_npmi')
coherence = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence)

**Diversity Score**

upload rbo.py file before importing

In [ ]:
import itertools
from rbo import rbo
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= round(inverted_rbo_calculator.irbo(topics_bert, topk=10, weight=0.9),2)
print("Inverted RBO Score:", IRBO)

# Visualize Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10,width=220, height=270, title="Topic Word Scores")

# Model serialization

In [ ]:
# Save model
topic_model.save("my_model")

2024-06-09 08:46:48,913 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
loaded_topic_model = topic_model.load("my_model")
loaded_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2470,0_لوگ_کوئی_لوگوں_دن,"[لوگ, کوئی, لوگوں, دن, جائیں, وقت, تمہیں, تمہا...",[اور اس کی نشانیوں میں سے یہ ہے کہ وہ تمہیں بج...
1,1,1764,1_رب_لوگوں_اے_لوگ,"[رب, لوگوں, اے, لوگ, ایمان, خدا, کوئی, تمہارے,...",[میں تو تمہارے رب پر ایمان لے آیا، تم بھی میری...
2,2,1719,2_اللہ_کوئی_لوگ_تمہارے,"[اللہ, کوئی, لوگ, تمہارے, لوگوں, ایمان, اے, تم...",[اور جب ان سے کہا جاتا ہے کہ اللہ نے جو رزق تم...
3,3,268,3_موسی_فرعون_قوم_نوح,"[موسی, فرعون, قوم, نوح, رب, اسرائیل, بنی, اے, ...","[موسیؑ اور ہارونؑ کے رب کو"", فرعون نے کہا "" اچ..."
4,4,15,4_کن_انعامات_جھٹلاؤ_رب,"[کن, انعامات, جھٹلاؤ, رب, , , , , , ]","[اپنے رب کے کن کن انعامات کو تم جھٹلاؤ گے؟, اپ..."


# LDA

We use the [ parallelized Latent Dirichlet Allocation (LDA)](https://radimrehurek.com/gensim/models/ldamulticore.html) from Gensim.

Note: for LDA you have to define topics number in advance.

In [ ]:
n_topics=5
lda = LdaMulticore(corpus, id2word=id2word, random_state=42, num_topics=5)
topics = lda.show_topics(num_topics=5, formatted=False)

#Extract the words from the topics
topics_list = []
for _, topic_words in topics:
    words = [word for word, _ in topic_words]
    topics_list.append(words)

print(topics_list)

In [ ]:
#CV Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_lda)

In [ ]:
#NPMI Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_lda = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_lda)

In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_LDA = round(inverted_rbo_calculator.irbo(topics_list, topk=10, weight=0.9),2)
print('\nIRBO Score: ',IRBO_LDA)

#NMF
We use Gensim library for implementation of NMF.

Note: for NMF you have to define topics number in advance.

In [ ]:
#Using Gensim
from gensim import corpora, models


# Train NMF model
num_topics = 5  # Define the number of topics
nmf_model = models.Nmf(corpus, num_topics=num_topics, id2word=id2word, random_state=42)

# Extract topics
topics = []
for topic_id in range(num_topics):
    topic_words = nmf_model.show_topic(topic_id, topn=10)  # You can adjust topn as needed
    topic_words = [word for word, _ in topic_words]
    topics.append(topic_words)

# Print topics in the desired format
print(topics)

In [ ]:
#Calculate CV Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_nmf = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_nmf)

In [ ]:
#Calculate NPMI Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_nmf = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_nmf)

In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_NMF = round(inverted_rbo_calculator.irbo(topics, topk=10, weight=0.9),2)
print('\nIRBO Score: ',IRBO_NMF)